In [4]:
import findspark
findspark.init('/opt/spark')

import os
from dotenv import load_dotenv
load_dotenv('../.env')
access = os.environ.get('AWS_ACCESS')
secret = os.environ.get('AWS_SECRET')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
install_pypi_packages can only use called when spark.pyspark.virtualenv.enabled is set to true
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/context.py", line 1178, in install_pypi_package
    pypi_package = self._validate_package(pypi_package)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/context.py", line 1245, in _validate_package
    raise RuntimeError("install_pypi_packages can only use called when "
RuntimeError: install_pypi_packages can only use called when spark.pyspark.virtualenv.enabled is set to true



In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.types import StringType
from pyspark.sql.functions import col, size, udf

conf = SparkConf() \
    .set("fs.s3a.awsAccessKeyId", access) \
    .set("fs.s3a.awsSecretAccessKey", secret) \
    .set("fs.s3a.endpoint", "s3.us-east-1.amazonaws.com") \
    .set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .set("fs.s3a.impl","org.apache.hadoop.fs.s3native.NativeS3FileSystem") \
    .set("com.amazonaws.services.s3.enableV4", "true") \
    .set("spark.hadoop.mapreduce.fileoutputcommitter.algorithm.version", "2") \
    .set("spark.hadoop.fs.s3a.fast.upload","true") \
    .set("spark.sql.parquet.filterPushdown", "true")

spark = SparkSession.builder.appName('data-cleaning').config(conf=conf).getOrCreate()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
3,application_1618254796607_0005,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
import sparknlp
from sparknlp.base import Finisher, DocumentAssembler
from sparknlp.annotator import (Tokenizer, Normalizer,
                                LemmatizerModel, StopWordsCleaner)
from sparknlp.pretrained import PretrainedPipeline
from pyspark.ml import Pipeline

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
filename = ''
df = spark.read.parquet(filename).drop('geo', 'coordinates', 'place', 'Unnamed: 0', 'Unnamed: 0.1')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- id: string (nullable = true)
 |-- full_text: string (nullable = true)
 |-- retweet_count: double (nullable = true)
 |-- favorite_count: double (nullable = true)

In [30]:
import preprocessor as p

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
def tweetPreprocessor(text):
    p.set_options(p.OPT.URL, p.OPT.EMOJI, p.OPT.SMILEY, p.OPT.NUMBER, p.OPT.MENTION)
    text = p.clean(text)
    return text
tweetPreprocessor_udf = udf(tweetPreprocessor, StringType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
df = df.withColumn('clean', tweetPreprocessor_udf('full_text'))
df.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-------------+--------------+--------------------+
|                  id|           full_text|retweet_count|favorite_count|               clean|
+--------------------+--------------------+-------------+--------------+--------------------+
|1.300070747059167...|@kmiranda1973 @Mi...|          0.0|           2.0|                    |
|1.300070747453427...|@realDonaldTrump ...|          0.0|           0.0|                    |
|1.300070747566755...|@realDonaldTrump ...|          0.0|           1.0|Pizza and Chicken...|
|1.300070748116131...|@EricTrump @realD...|          0.0|           1.0|                    |
|1.300070748359454...|Impeached @realdo...|          0.0|           1.0|Impeached and #mi...|
+--------------------+--------------------+-------------+--------------+--------------------+
only showing top 5 rows

In [33]:
# The nltk stopword won't download onto emr without some setup so it's just easier to copy and paste
stopwords_list = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:

documentAssembler = DocumentAssembler() \
    .setInputCol('clean') \
    .setOutputCol('document') \
    .setCleanupMode('shrink_full')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')

stopwords_cleaner = StopWordsCleaner() \
    .setInputCols(['token']) \
    .setOutputCol('removed_stop') \
    .setCaseSensitive(False) \
    .setStopWords(stopwords_list)

normalizer = Normalizer() \
    .setInputCols(['removed_stop']) \
    .setOutputCol('normalized') \
    .setLowercase(True)

lemmatizer = LemmatizerModel.pretrained() \
    .setInputCols(['normalized']) \
    .setOutputCol('lemma')

finisher = Finisher() \
     .setInputCols(['lemma']) \
     .setCleanAnnotations(True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]

In [35]:
pipeline = Pipeline() \
     .setStages([
           documentAssembler,
           tokenizer,
           stopwords_cleaner,
           normalizer,
           lemmatizer,
           finisher
     ])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
df = pipeline.fit(df).transform(df)
df = df.drop('clean').withColumnRenamed('finished_lemma', 'clean_text')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-------------+--------------+--------------------+
|                  id|           full_text|retweet_count|favorite_count|          clean_text|
+--------------------+--------------------+-------------+--------------+--------------------+
|1.300070747059167...|@kmiranda1973 @Mi...|          0.0|           2.0|                  []|
|1.300070747453427...|@realDonaldTrump ...|          0.0|           0.0|                  []|
|1.300070747566755...|@realDonaldTrump ...|          0.0|           1.0|[pizza, chicken, ...|
|1.300070748116131...|@EricTrump @realD...|          0.0|           1.0|                  []|
|1.300070748359454...|Impeached @realdo...|          0.0|           1.0|[impeach, mikepen...|
|1.300070748367786...|@PollWatch2020 @r...|          0.0|           0.0|      [head, advice]|
|1.300070748631916...|@RichLowry @realD...|          0.0|           0.0|[kind, like, glos...|
|1.300070748942344...|@realDonaldTrump ...|          0.0|   

In [37]:
df = df.where(size(col('clean_text')) > 4)
df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------------------+-------------+--------------+--------------------+
|                  id|           full_text|retweet_count|favorite_count|          clean_text|
+--------------------+--------------------+-------------+--------------+--------------------+
|1.300070748359454...|Impeached @realdo...|          0.0|           1.0|[impeach, mikepen...|
|1.300070748631916...|@RichLowry @realD...|          0.0|           0.0|[kind, like, glos...|
|1.300070748942344...|@realDonaldTrump ...|          0.0|           1.0|[let, start, pros...|
|1.300070749336727...|@eugenegu @realDo...|          0.0|           0.0|[ok, let, talk, t...|
|1.300070751903645...|@ElvisKirby4 @jas...|          0.0|           0.0|[fallacy, violati...|
|1.300070752104992...|@realDonaldTrump ...|          0.0|           0.0|[nobody, give, sh...|
|1.300070752763347...|People who don’t ...|          0.0|           0.0|[people, dont, ev...|
|1.300070752788545...|@RacismWoke @manp...|          0.0|   

Writing will take forever

In [38]:
filename = ''
df.write.parquet(filename, mode='overwrite')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…